# tutorial 2: ensemble learning

This tutorial shows you how to use classes from the **ensemble_learning** module to:
* make predictions from an ensemble of different algorithms trained on the same input channel using the **Ensemble** class
* make predictions from an ensemble of models trained on different input channels using the **ChanneEnsemble** class

Competition-winning models often use ensemble learning approaches that combine the inferences of an ensemble of different ML algorithms to make a final prediction.  Predictor voting and stacked generalization are powerful methods for combining inferences that can create accurate predictions from less accurate models. Both methods require that prediction error in the ensemble is uncorrelated.

Pipecaster makes conventional single channel voting and stacked genearlization approaches easy with a single class: **Ensemble**.  In addition, the **ChannelEnsemble** class provides similar functionality for models trained on different input channels.

Both **Ensemble** and **ChannelEnsemble** classes also allow screening of base predictors using internal cross validation.  Screening can sometimes improve performance by dropping inaccurate models.

## Single channel ensembles

### Stacked generalization

This example illustrates the use of pipecaster's **Ensemble** class to stack ML models for ensemble learning.  The ensemble in this example has six different ML classifier algorithms.  A support vector machine (SVC) is stacked on top of these base classifiers, using their predictions as features for meta-prediction.

**Internal Cross Validation (CV) Training**
When training a meta-predictor (SVC in this example), it is standard practice to use internal
CV training of the base classifiers to prevent them from making inferences on training samples (1).  With internal CV training, training sample prediction is avoided by training training each base predictor multiple times on subsets of the data and using the models to make predicitions about held-out samples.  Line 23 in the code below activates internal CV and line 24 specifies that the CV prediction will be used to train the meta-predictor (the default option when CV is activated, shown here to be explicit).

(1) Wolpert, David H. "Stacked generalization."
Neural networks 5.2 (1992): 241-259.

In [4]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pipecaster as pc

X, y = make_classification(n_classes=2, n_samples=500, n_features=100,
                           n_informative=5, class_sep=0.6)

predictors = [MLPClassifier(), LogisticRegression(),
              KNeighborsClassifier(),  GradientBoostingClassifier(),
              RandomForestClassifier(), GaussianNB()]

ensemble_clf = pc.Ensemble(
                 base_predictors=predictors,
                 meta_predictor=SVC(),
                 internal_cv=5,
                 disable_cv_train=False, 
                 base_processes='max')

clf = Pipeline([('scaler', StandardScaler()),
                ('ensemble_clf', ensemble_clf)])
pc.cross_val_score(clf, X, y)

(pid=78785) /Users/john/trading/src/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
(pid=78785)   % self.max_iter, ConvergenceWarning)
(pid=78784) /Users/john/trading/src/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
(pid=78784)   % self.max_iter, ConvergenceWarning)


[0.783715925394548, 0.7706599713055955, 0.7697893972403775]

### Voting

This example illustrates the use of pipecaster's **Ensemble** class to make a voting ensemble classifier.  The ensemble in this example has six different ML classifier algorithms.  **SoftVotingMetaClassifer** takes the prediction probabilites from these classifiers uses argmax used to chose a class.  Pipecaster also provides a **HardVotingMetaClassifier** class that predicts with the most frequent base prediction, and the **AggregatingMetaRegressor** class to combine the predictions of regressors.

*Note*: Pipecaster's single channele classes, like Ensemble, should be fully compatible with scikit-learn.  For instance, in this example the Ensemble is used in a scikit-learn Pipeline object. 

In [5]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
import pipecaster as pc

X, y = make_classification(n_classes=2, n_samples=500, n_features=100,
                           n_informative=5, class_sep=0.6)

predictors = [MLPClassifier(), LogisticRegression(),
              KNeighborsClassifier(),  GradientBoostingClassifier(),
              RandomForestClassifier(), GaussianNB()]

ensemble_clf = pc.Ensemble(
                 base_predictors=predictors,
                 meta_predictor=pc.SoftVotingMetaClassifier(),
                 base_processes='max')

clf = Pipeline([('scaler', StandardScaler()), ('ensemble_clf', ensemble_clf)])
pc.cross_val_score(clf, X, y)

[0.7127796901893287, 0.694635685599541, 0.7110917537746806]

### Stacked generalization with model selection

Internal cross validation was introduced in the first example in this notebook as a method for reducing overfitting during stacked generalization.  Pipecaster also takes advantage of internal cross validation training to estimate model performance for the purpose of in-pipeline model selection.  Inaccurate models in the ensemble can sometime reduce the performance of your pipeline.  

Lines 25-27 in this example are the only lines that differ from the first example.  These lines are added to provide methods for scoring and selecting base models.  Line 25 sets the method used by the base classifiers for making predictions when their performance is being assessed.  Line 26 sets the performance metric.  Line 27 instructs Ensemble to select the 3 models with the highest balanced_accuracy scores.  For more information see the the API documentation for **Ensemble** and for pipecaster's **score_selection** module.   

In [3]:
from sklearn.datasets import make_classification
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pipecaster as pc

X, y = make_classification(n_classes=2, n_samples=500, n_features=100,
                           n_informative=5, class_sep=0.6)

predictors = [MLPClassifier(), LogisticRegression(),
              KNeighborsClassifier(),  GradientBoostingClassifier(),
              RandomForestClassifier(), GaussianNB()]

ensemble_clf = pc.Ensemble(
                 base_predictors=predictors,
                 meta_predictor=SVC(),
                 internal_cv=5, 
                 base_score_methods='predict',
                 scorer=balanced_accuracy_score,
                 score_selector=pc.RankScoreSelector(k=3),
                 disable_cv_train=False, 
                 base_processes='max')

clf = Pipeline([('scaler', StandardScaler()),
                ('ensemble_clf', ensemble_clf)])

pc.cross_val_score(clf, X, y, score_method='predict', scorer=balanced_accuracy_score)

(pid=79767) /Users/john/trading/src/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
(pid=79767)   % self.max_iter, ConvergenceWarning)
(pid=79767) /Users/john/trading/src/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
(pid=79767)   % self.max_iter, ConvergenceWarning)
(pid=79765) /Users/john/trading/src/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
(pid=79765)   % self.max_iter, ConvergenceWarning)
(pid=79764) /Users/john/trading/src/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: C

[0.7304216867469879, 0.832903040734366, 0.7530120481927711]

In [4]:
clf.fit(X, y)
ensemble_clf = clf.named_steps['ensemble_clf']
ensemble_clf.get_screen_results()

(pid=79766) /Users/john/trading/src/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
(pid=79766)   % self.max_iter, ConvergenceWarning)
(pid=79766) /Users/john/trading/src/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
(pid=79766)   % self.max_iter, ConvergenceWarning)
(pid=79766) /Users/john/trading/src/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
(pid=79766)   % self.max_iter, ConvergenceWarning)
(pid=79766) /Users/john/trading/src/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: C

,performance,selections
model,,
MLPClassifier(),0.600,-
LogisticRegression(),0.566,-
KNeighborsClassifier(),0.554,-
GradientBoostingClassifier(),0.740,+++
RandomForestClassifier(),0.708,+++
GaussianNB(),0.714,+++


### Model selection without ensemble prediction

The **Ensemble** class can also be used without a meta_predictor for model model selection.  When the meta_predictor parameter is left at the default value of None during Ensemble initialization, Ensemble will use internal cv and scoring to select the best predictor in the ensemble during fitting.  Only the most accurate predictor in the ensemble predictor will be stored and used for inference.

In [5]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
import pipecaster as pc

X, y = make_classification(n_classes=2, n_samples=500, n_features=100,
                           n_informative=5, class_sep=0.6)

predictors = [MLPClassifier(), LogisticRegression(),
              KNeighborsClassifier(),  GradientBoostingClassifier(),
              RandomForestClassifier(), GaussianNB()]

ensemble_clf = pc.Ensemble(
                 base_predictors=predictors,
                 meta_predictor=None,
                 internal_cv=5, 
                 base_score_methods='predict',
                 scorer=balanced_accuracy_score,
                 base_processes='max')

clf = Pipeline([('scaler', StandardScaler()),
                ('ensemble_clf', ensemble_clf)])

pc.cross_val_score(clf, X, y, score_method='predict', scorer=balanced_accuracy_score)

(pid=79767) /Users/john/trading/src/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
(pid=79767)   % self.max_iter, ConvergenceWarning)
(pid=79765) /Users/john/trading/src/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
(pid=79765)   % self.max_iter, ConvergenceWarning)


[0.7964716006884682, 0.7964716006884682, 0.8072289156626506]

In [6]:
clf.fit(X, y)
ensemble_clf = clf.named_steps['ensemble_clf']
ensemble_clf.get_screen_results()

,performance,selections
model,,
MLPClassifier(),0.740208,-
LogisticRegression(),0.748656,-
KNeighborsClassifier(),0.665704,-
GradientBoostingClassifier(),0.849392,+++
RandomForestClassifier(),0.805768,-
GaussianNB(),0.780576,-


## Multichannel ensembles

The **ChannelEnsemble** class supports the same functionality as **Ensemble** (voting, stacked generalization, model selection) but instead of training an ensemble of models for a single channel, it trains an ensemble of models for multiple channels with one model per channel.  Training an ML model on each input and combining inferences through voting or stacking can improve predictive accuracy over a simple concatenation -> ML pipeline.  

*Note*:  You can build multichannel ensembles manually without the **ChannelEnsemble** class using using the **transform_wrappers** module to convert predictors into transformers and provide internal cv training functionality, **ChannelConcenator** to combine outputs from the base predictors, and **MultichannelPipeline** layering to stack the concatenator and meta-predictor.  For more info on this usage style, see the 3 different style examples in the **MultichannelPipeline** and **SoftVotingMetaClassifier** API documenation (or docstrings).

### Stacked generalization

In [29]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import pipecaster as pc

Xs, y, X_types = pc.make_multi_input_classification(n_informative_Xs=3,
                                                    n_random_Xs=7)

clf = pc.MultichannelPipeline(n_channels=10)
clf.add_layer(StandardScaler())
clf.add_layer(pc.ChannelEnsemble(GradientBoostingClassifier(), SVC(), internal_cv=5),
              pipe_processes='max')

pc.cross_val_score(clf, Xs, y)

[0.9117647058823529, 0.96875, 0.9393382352941176]

In [30]:
clf.fit(Xs, y)

,layer_0,out_0,layer_1,out_1
channel,,,,
0,StandardScaler,→,ChannelEnsemble,→
1,StandardScaler,→,▽,
2,StandardScaler,→,▽,
3,StandardScaler,→,▽,
4,StandardScaler,→,▽,
5,StandardScaler,→,▽,
6,StandardScaler,→,▽,
7,StandardScaler,→,▽,
8,StandardScaler,→,▽,


### Voting

In [25]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
import pipecaster as pc

Xs, y, X_types = pc.make_multi_input_classification(n_informative_Xs=3,
                                                    n_random_Xs=7)

clf = pc.MultichannelPipeline(n_channels=10)
clf.add_layer(StandardScaler())
clf.add_layer(pc.ChannelEnsemble(GradientBoostingClassifier(), pc.SoftVotingMetaClassifier()),
              pipe_processes='max')

pc.cross_val_score(clf, Xs, y)

[0.8823529411764706, 0.8492647058823529, 0.8768382352941176]

In [26]:
clf.fit(Xs, y)

,layer_0,out_0,layer_1,out_1
channel,,,,
0,StandardScaler,→,ChannelEnsemble,→
1,StandardScaler,→,▽,
2,StandardScaler,→,▽,
3,StandardScaler,→,▽,
4,StandardScaler,→,▽,
5,StandardScaler,→,▽,
6,StandardScaler,→,▽,
7,StandardScaler,→,▽,
8,StandardScaler,→,▽,


### Stacked generalization with model selection

In [4]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import pipecaster as pc

Xs, y, X_types = pc.make_multi_input_classification(n_informative_Xs=3,
                                                    n_random_Xs=7)

clf = pc.MultichannelPipeline(n_channels=10)
clf.add_layer(StandardScaler())
clf.add_layer(pc.ChannelEnsemble(GradientBoostingClassifier(), SVC(), internal_cv=5, 
                 scorer='auto',
                 score_selector=pc.RankScoreSelector(k=3),),
              pipe_processes='max')

pc.cross_val_score(clf, Xs, y)

[0.9705882352941176, 0.9411764705882353, 0.9705882352941176]

In [5]:
clf.fit(Xs, y)

,layer_0,out_0,layer_1,out_1
channel,,,,
0,StandardScaler,→,ChannelEnsemble,→
1,StandardScaler,→,▽,
2,StandardScaler,→,▽,
3,StandardScaler,→,▽,
4,StandardScaler,→,▽,
5,StandardScaler,→,▽,
6,StandardScaler,→,▽,
7,StandardScaler,→,▽,
8,StandardScaler,→,▽,


In [6]:
ensemble_clf = clf.get_model(1,0)
df = ensemble_clf.get_screen_results()
df['input type'] = X_types
df

,performance,selections,input type
channel,,,
0,0.42,-,random
1,0.56,-,random
2,0.84,+++,informative
3,0.92,+++,informative
4,0.58,-,random
5,0.45,-,random
6,0.56,-,random
7,0.86,+++,informative
8,0.58,-,random


### Model selection

In [1]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import pipecaster as pc

Xs, y, X_types = pc.make_multi_input_classification(n_informative_Xs=3,
                                                    n_random_Xs=7)

clf = pc.MultichannelPipeline(n_channels=10)
clf.add_layer(StandardScaler())
clf.add_layer(pc.ChannelEnsemble(GradientBoostingClassifier(), internal_cv=5, scorer='auto'),
              pipe_processes='max')

pc.cross_val_score(clf, Xs, y)

[0.9411764705882353, 0.9705882352941176, 0.7224264705882353]

In [2]:
clf.fit(Xs, y)

,layer_0,out_0,layer_1,out_1
channel,,,,
0,StandardScaler,→,ChannelEnsemble,→
1,StandardScaler,→,▽,
2,StandardScaler,→,▽,
3,StandardScaler,→,▽,
4,StandardScaler,→,▽,
5,StandardScaler,→,▽,
6,StandardScaler,→,▽,
7,StandardScaler,→,▽,
8,StandardScaler,→,▽,


In [3]:
ensemble_clf = clf.get_model(1,0)
df = ensemble_clf.get_screen_results()
df['input type'] = X_types
df

,performance,selections,input type
channel,,,
0,0.569628,-,random
1,0.508403,-,random
2,0.479792,-,random
3,0.519808,-,random
4,0.729692,-,informative
5,0.748900,+++,informative
6,0.699480,-,informative
7,0.510004,-,random
8,0.509604,-,random
